In [4]:
pip install requests beautifulsoup4


In [12]:
import requests
import os
import time
import random

# -----------------------------
# Clear screen
# -----------------------------
def clear():
    os.system("cls" if os.name == "nt" else "clear")

# -----------------------------
# Stat bar function
# -----------------------------
def stat_bar(label, value, danger_high=False):
    filled = "█" * value
    empty = "░" * (10 - value)

    if danger_high:  # For Hunger (higher = worse)
        if value >= 8:
            status = "Starving!"
        elif value <= 4:
            status = "Full"
        else:
            status = "Moderate"
    else:  # Happiness / Energy (higher = better)
        if value >= 7:
            status = "Good"
        elif value <= 3:
            status = "Low"
        else:
            status = "Moderate"

    print(f"{label:<10} {filled}{empty} {value}/10  {status}")

# -----------------------------
# WEATHER
# -----------------------------
def get_weather(zip_code):
    try:
        url = f"https://wttr.in/{zip_code}?format=j1"
        data = requests.get(url, timeout=5).json()

        area = data["nearest_area"][0]["areaName"][0]["value"]
        temp = data["current_condition"][0]["temp_F"]
        desc = data["current_condition"][0]["weatherDesc"][0]["value"]

        mood_effect = 0
        energy_effect = 0
        effect_text = "Neutral effect."

        if "sun" in desc.lower():
            mood_effect += 2
            energy_effect += 1
            effect_text = "Sunny weather boosts happiness and energy."
        elif "rain" in desc.lower() or "storm" in desc.lower():
            mood_effect -= 2
            effect_text = "Rainy or stormy weather lowers happiness."
        elif "snow" in desc.lower():
            mood_effect -= 1
            effect_text = "Cold or snowy weather slightly lowers energy."

        return {
            "city": area,
            "temp": temp,
            "desc": desc,
            "mood": mood_effect,
            "energy": energy_effect,
            "effect_text": effect_text
        }
    except:
        return None

# -----------------------------
# BITCOIN
# -----------------------------
def get_bitcoin():
    try:
        url = "https://api.coinbase.com/v2/prices/spot?currency=USD"
        data = requests.get(url, timeout=5).json()
        price = float(data["data"]["amount"])

        if price > 65000:
            effect = +2
            status = "Bull Market 📈"
            effect_text = "Bitcoin surging! Happiness increases."
        elif price < 45000:
            effect = -2
            status = "Bear Market 📉"
            effect_text = "Bitcoin falling. Pet feels stressed."
        else:
            effect = 0
            status = "Stable Market ⚖️"
            effect_text = "Bitcoin is stable. No effect on pet."

        return price, status, effect, effect_text
    except:
        return None, "Unavailable", 0, "Bitcoin data unavailable."

# -----------------------------
# NBA
# -----------------------------
def get_nba_team(team_name):
    try:
        url = "https://site.web.api.espn.com/apis/v2/sports/basketball/nba/scoreboard"
        data = requests.get(url, timeout=5).json()

        for game in data["events"]:
            for comp in game["competitions"]:
                teams = comp["competitors"]
                for t in teams:
                    if team_name.lower() == t["team"]["displayName"].lower():
                        opponent = [x for x in teams if x != t][0]
                        team_score = int(t["score"])
                        opp_score = int(opponent["score"])
                        if team_score > opp_score:
                            return "WIN 🏆", +2, f"{team_name} won the last game! Happiness increases."
                        else:
                            return "LOSS 💔", -2, f"{team_name} lost the last game. Happiness decreases."
        return "No recent game", 0, "No recent NBA game data. Neutral effect."
    except:
        return "Unavailable", 0, "NBA data unavailable. Neutral effect."

# -----------------------------
# MAIN GAME LOOP
# -----------------------------
def play_game():
    clear()

    # Pet selection
    animal_emojis = ["🐶","🐱","🐰","🐹","🦊","🐻","🐼","🐸","🐵","🐧"]
    print("Choose your pet emoji:")
    for i, e in enumerate(animal_emojis, 1):
        print(f"{i}. {e}")
    emoji_choice = int(input("Pick 1-10: ")) - 1
    pet_emoji = animal_emojis[emoji_choice]

    pet_name = input("Name your pet: ")

    # Favorite toy
    toys = ["🧸 Teddy", "⚽ Ball", "🪃 Boomerang", "🧶 Yarn", "🦴 Bone"]
    print("\nChoose favorite toy:")
    for i, t in enumerate(toys, 1):
        print(f"{i}. {t}")
    fav_toy = toys[int(input("Pick 1-5: ")) - 1]

    zip_code = input("\nEnter ZIP code: ")
    nba_team = input("Enter full NBA team name (ex: Boston Celtics): ")

    # Stats
    hunger = 5
    happiness = 5
    energy = 5

    while True:
        clear()
        weather = get_weather(zip_code)
        btc_price, btc_status, btc_effect, btc_text = get_bitcoin()
        nba_result, nba_effect, nba_text = get_nba_team(nba_team)

        # Apply effects
        if weather:
            happiness += weather["mood"]
            energy += weather["energy"]
        happiness += btc_effect
        happiness += nba_effect

        # Clamp stats
        hunger = min(max(hunger,0),10)
        happiness = min(max(happiness,0),10)
        energy = min(max(energy,0),10)

        # Dashboard
        print(f"{pet_emoji} {pet_name}")
        stat_bar("Hunger", hunger, danger_high=True)
        stat_bar("Happiness", happiness)
        stat_bar("Energy", energy)
        print()

        if weather:
            print(f"🌤 Weather in {weather['city']}: {weather['temp']}°F, {weather['desc']}")
            print(f"Effect: {weather['effect_text']}\n")
        else:
            print("🌤 Weather unavailable\n")

        if btc_price:
            print(f"₿ Bitcoin: ${btc_price:,.2f} ({btc_status})")
            print(f"Effect: {btc_text}\n")
        else:
            print("₿ Bitcoin unavailable\n")

        print(f"🏀 NBA ({nba_team}): {nba_result}")
        print(f"Effect: {nba_text}\n")

        # Actions
        print("Actions:")
        print("1. Feed 🍎")
        print("2. Play 🎾")
        print("3. Sleep 😴")
        print("4. Medicine 💊")
        print("5. Quit")

        choice = input("Choose: ")

        if choice == "1":
            prev = hunger
            hunger = max(0,hunger-2)
            print(f"\nYou just fed {pet_name}, Hunger decreased by {prev - hunger}.")
            time.sleep(1)
        elif choice == "2":
            toy = random.choice(toys)
            prev = happiness
            if toy == fav_toy:
                happiness = min(10,happiness+2)
                print(f"\nPlayed with {toy} (favorite!) Happiness +{happiness-prev}")
            else:
                happiness = min(10,happiness+1)
                print(f"\nPlayed with {toy}, Happiness +{happiness-prev}")
            energy = max(0,energy-1)
            time.sleep(1)
        elif choice == "3":
            prev_energy = energy
            energy = min(10,energy+3)
            hunger = min(10,hunger+1)
            print(f"\n{pet_name} slept, Energy +{energy-prev_energy}, Hunger +1")
            time.sleep(1)
        elif choice == "4":
            prev = happiness
            happiness = min(10,happiness+1)
            print(f"\nYou gave medicine to {pet_name}, Happiness +{happiness-prev}")
            time.sleep(1)
        elif choice == "5":
            break

    print("\nThanks for playing!")

# -----------------------------
play_game()


Choose your pet emoji:
1. 🐶
2. 🐱
3. 🐰
4. 🐹
5. 🦊
6. 🐻
7. 🐼
8. 🐸
9. 🐵
10. 🐧
Pick 1-10: 1
Name your pet: re

Choose favorite toy:
1. 🧸 Teddy
2. ⚽ Ball
3. 🪃 Boomerang
4. 🧶 Yarn
5. 🦴 Bone
Pick 1-5: 1

Enter ZIP code: 33432
Enter full NBA team name (ex: Boston Celtics): Boston Celtics
🐶 re
Hunger     █████░░░░░ 5/10  Moderate
Happiness  ███████░░░ 7/10  Good
Energy     █████░░░░░ 5/10  Moderate

🌤 Weather in Boca Raton: 72°F, Partly cloudy
Effect: Neutral effect.

₿ Bitcoin: $89,757.99 (Bull Market 📈)
Effect: Bitcoin surging! Happiness increases.

🏀 NBA (Boston Celtics): Unavailable
Effect: NBA data unavailable. Neutral effect.

Actions:
1. Feed 🍎
2. Play 🎾
3. Sleep 😴
4. Medicine 💊
5. Quit
Choose: 1

You just fed re, Hunger decreased by 2.
🐶 re
Hunger     ███░░░░░░░ 3/10  Full
Happiness  █████████░ 9/10  Good
Energy     █████░░░░░ 5/10  Moderate

🌤 Weather in Boca Raton: 72°F, Partly cloudy
Effect: Neutral effect.

₿ Bitcoin: $89,770.06 (Bull Market 📈)
Effect: Bitcoin surging! Happiness incr